In [1]:
import pandas as pd

In [2]:
!pwd

/Users/ezb/Lambda/DS_Model_Engineering/Wrangling/Intermediate_data


In [3]:
# import datasets
allbuds = pd.read_csv('allbuds_strain_data_rename.csv')
kaggle = pd.read_csv('cannabis_kaggle_rename.csv')

In [4]:
print(allbuds.shape)
allbuds.head()

(5449, 8)


,strain_name,strain_desc,strain_type,strain_percent_indica,strain_percent_sativa,strain_thc_percent,strain_flavors,strain_effects
0,sugar-cane,Sugar Cane is a rare slightly sativa dominant ...,Sativa Dominant Hybrid,40% Indica,60% Sativa,20.0,Earthy Sweet Candy Grape Spicy Fruity Herbal P...,Body High Cerebral Creative Energizing Relaxin...
1,mac1,"MAC 1, also known as “Miracle Alien Cookies X1...",Hybrid,50% Indica,50% Sativa,21.5,Sweet Diesel Sour Spicy Herbal Pungent,Creative Euphoria Happy Motivation Relaxing Up...
2,chemdawg,With a near-even balance between sativa and in...,Indica Dominant Hybrid,55% Indica,45% Sativa,19.0,Earthy Pungent Chemical Diesel Pine,Cerebral Creative Euphoria Happy Relaxing Cere...
3,jack-herer,Jack Herer is easily one of the best-known str...,Sativa,NaN,NaN,23.0,Earthy Sweet Spicy Herbal Lemon Pine Woody,Body High Cerebral Creative Euphoria Happy Bod...
4,nerds,"Nerds, also known as “Nerdz,” is an evenly bal...",Hybrid,50% Indica,50% Sativa,15.5,Earthy Sweet Grape Spicy Herbal Fruity Berry W...,Cerebral Creative Euphoria Focus Relaxing Cere...


In [5]:
print(kaggle.shape)
kaggle.head()

(2351, 6)


,Strain,Type,Rating,Effects,Flavor,Description
0,100-og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-white-widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24k-gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [6]:
# rename columns to match kaggle dataset for merging
allbuds = allbuds.rename(columns={'strain_name':'Strain',
                                  'strain_desc':'Description',
                                  'strain_type':'Type',
                                  'strain_percent_indica':'Percent Indica',
                                  'strain_percent_sativa':'Percent Sativa',
                                  'strain_thc_percent':'THC Percent',
                                  'strain_flavors':'Flavor',
                                  'strain_effects':'Effects'})

In [8]:
# stripped string from Percent Indica, Percent Sativa and THC Percent
# and converted to float. Also allbuds Types column specifies whether
# a hybrid strain is sativa or indica dominant, removed specification
# to just say hybrid.

def snc2f(data):
    data = data.copy()
    p_indica = []
    p_sativa = []
    p_thc = []
    types = []
    
    for row in data['Percent Indica']:
        if isinstance(row, str) == True:
            stripped = row.strip('% Indica')
            converted = float(stripped)/100
            p_indica.append(converted)
        else:
            p_indica.append(float('nan'))

    for row in data['Percent Sativa']:
        if isinstance(row, str) == True:
            stripped = row.strip('% Sativa')
            converted = float(stripped)/100
            p_sativa.append(converted)
        else:
            p_sativa.append(float('nan'))

    for row in data['THC Percent']:
        if isinstance(row, float) == True:
            converted = row/100
            p_thc.append(converted)
        else:
            p_thc.append(float('nan'))

    for row in data['Type']:
        row = row.lower()
        if 'hybrid' in row:
            types.append('hybrid')
        else:
            types.append(row)

    allbuds['Percent Indica'] = p_indica
    allbuds['Percent Sativa'] = p_sativa
    allbuds['THC Percent'] = p_thc
    allbuds['Type'] = types
    
    return 
    
allbuds = snc2f(allbuds)

In [9]:
print(allbuds.shape)
allbuds.head()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# checking for duplicated strain names
allbuds[allbuds['Strain'].duplicated() == True]

In [ ]:
# dropped duplicates
allbuds = allbuds.drop(allbuds.index[[5121, 5258, 5351]])

In [ ]:
# checking for duplicated strain names
kaggle[kaggle['Strain'].duplicated() == True]

In [ ]:
# dropped duplicates
kaggle = kaggle.drop_duplicates()

In [ ]:
print(allbuds.shape)
kaggle.shape

In [ ]:
# initial merge of 2 datasets
merged = pd.merge(allbuds, kaggle, on=['Strain'], how='outer', validate='1:1', suffixes=('_allbuds', '_leafly'))

In [10]:
print(merged.shape)
merged.head()

NameError: name 'merged' is not defined

In [ ]:
merged.isna().sum()

In [ ]:
# function to remove double columns on Description, Type, Flavor, and Effects
def clean_dataset(data):
    df = pd.DataFrame(columns=['Strain', 'Type', 'Percent Indica', 'Percent Sativa',
                               'THC Percent', 'Description 1', 'Description 2',
                               'Generated Description', 'Flavor', 'Effects',
                               'Rating'])
    description_1 = []
    description_2 = []
    types = [] 
    flavor = []
    flavor_leafly = []
    effects = []
    effects_leafly = []
    
    
    for row in zip(data['Type_allbuds'], data['Type_leafly']):
        if row[0] == row[1]:
            types.append(row[0])
        elif (type(row[0]) == str) & (type(row[1]) == float):
            types.append(row[0])
        elif (type(row[1]) == str) & (type(row[0]) == float):
            types.append(row[1])
        elif (type(row[1]) == float) & (type(row[0]) == float):
            types.append(float('nan'))
        else:
            types.append('hybrid')
    
    for row in zip(data['Description_allbuds'], data['Description_leafly']):
        if (type(row[0]) == str) & (type(row[1]) == str):
            description_1.append(row[0])
            description_2.append(row[1])
        elif (type(row[0]) == str) & (type(row[1]) == float):
            description_1.append(row[0])
            description_2.append(float('nan'))
        elif (type(row[0]) == float) & (type(row[1]) == str):
            description_1.append(row[1])
            description_2.append(float('nan'))
        else:
            description_1.append(float('nan'))
            description_2.append(float('nan'))
            
    for row in data['Flavor_leafly']:
        if type(row) == str:
            row = row.replace(',', ' ')
            flavor_leafly.append(row)
        else:
            flavor_leafly.append(float('nan'))
    
    data['Flavor_leafly'] = flavor_leafly

    for row in zip(data['Flavor_allbuds'], data['Flavor_leafly']):
        if (type(row[0]) == str) & (type(row[1]) == str):
            flavor.append(str(row[0] + ' ' + row[1]))
        elif (type(row[0]) == str) & (type(row[1]) == float):
            flavor.append(row[0])
        elif (type(row[0]) == float) & (type(row[1]) == str):
            flavor.append(row[1])
        else:
            flavor.append(float('nan'))
    
    for row in data['Effects_leafly']:
        if type(row) == str:
            row = row.replace(',', ' ')
            effects_leafly.append(row)
        else:
            effects_leafly.append(float('nan'))
    
    data['Flavor_leafly'] = effects_leafly
    
    for row in zip(data['Effects_allbuds'], data['Effects_leafly']):
        if (type(row[0]) == str) & (type(row[1]) == str):
            effects.append(str(row[0] + ' ' + row[1]))
        elif (type(row[0]) == str) & (type(row[1]) == float):
            effects.append(row[0])
        elif (type(row[0]) == float) & (type(row[1]) == str):
            effects.append(row[1])
        else:
            effects.append(float('nan'))
    
    
    df['Strain'] = data['Strain']
    df['Description 1'] = description_1
    df['Description 2'] = description_2
    df['Type'] = types
    df['Flavor'] = flavor
    df['Effects'] = effects
    df['Rating'] = data['Rating']
    df['Percent Indica'] = data['Percent Indica']
    df['Percent Sativa'] = data['Percent Sativa']
    df['THC Percent'] = data['THC Percent']
    
    return df

In [ ]:
# final dataset
final = clean_dataset(merged)

In [ ]:
print(final.shape)
final.head()

In [ ]:
final.isna().sum()

In [ ]:
final.to_csv('processed_kaggle_allbuds.csv', index=False)